In [1]:
# This code identifies the coldest day of the year, the maximum consumption during a 3-hour period, and the internal and external temperatures at that time.

import os
import pandas as pd

# Path to the folder containing the CSV files
folder_path = "DATA"

# Prepare a list to store the results
results = []

# Define the autumn and winter months in Great Britain
autumn_winter_months = [9, 10, 11, 12, 1, 2, 3]

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        
        # Load the CSV file
        data = pd.read_csv(file_path)
        
        # Check if necessary columns are present
        required_columns = ['Timestamp', 'Whole_System_Energy_Consumed', 'External_Air_Temperature', 'Internal_Air_Temperature']
        if not all(col in data.columns for col in required_columns):
            print(f"Skipping file {filename} due to missing required columns.")
            continue
        
        # Convert the Timestamp column to a datetime object
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        
        # Sort data by Timestamp
        data = data.sort_values(by='Timestamp')
        
        # Convert columns to numeric, coercing errors
        data['External_Air_Temperature'] = pd.to_numeric(data['External_Air_Temperature'], errors='coerce')
        data['Internal_Air_Temperature'] = pd.to_numeric(data['Internal_Air_Temperature'], errors='coerce')
        data['Whole_System_Energy_Consumed'] = pd.to_numeric(data['Whole_System_Energy_Consumed'], errors='coerce')

        # Calculate the time difference in seconds between each measurement
        data['Time_Diff'] = data['Timestamp'].diff().dt.total_seconds()
        
        # Calculate the incremental energy consumption of the whole system
        data['Whole_System_Energy_Consumed_Incremental'] = data['Whole_System_Energy_Consumed'].diff()
        
        # Calculate the power in kW for each measurement period
        data['Whole_System_Energy_Consumed_Incremental_kW'] = data['Whole_System_Energy_Consumed_Incremental'] / (data['Time_Diff'] / 3600)
        
        # Drop rows with NaN values or zero consumption in the relevant columns
        data = data.dropna(subset=['Whole_System_Energy_Consumed_Incremental_kW', 'Whole_System_Energy_Consumed_Incremental', 'Internal_Air_Temperature'])
        data = data[(data['Whole_System_Energy_Consumed_Incremental_kW'] != 0) & 
                    (data['Whole_System_Energy_Consumed_Incremental'] != 0) &
                    (data['Internal_Air_Temperature'] != 0)]
        
        # Ensure that there are still rows remaining after dropping NaNs and zeros
        if data.empty:
            print(f"Skipping file {filename} due to insufficient data after processing.")
            continue

        # Add a column for the month extracted from the timestamp
        data['Month'] = data['Timestamp'].dt.month
        
        # Filter the data for autumn and winter months
        autumn_winter_data = data[data['Month'].isin(autumn_winter_months)]
        
        # Skip if no autumn/winter data with non-zero consumption found
        if autumn_winter_data.empty:
            print(f"Skipping file {filename} as no autumn/winter data with non-zero consumption found.")
            continue

        # Identify the coldest moment where consumption is not zero
        coldest_index = autumn_winter_data['External_Air_Temperature'].idxmin()
        
        # Get the timestamp of the coldest moment
        coldest_time = autumn_winter_data.loc[coldest_index, 'Timestamp']
        
        # Filter the data for that specific day
        coldest_day = coldest_time.date()
        day_data = data[data['Timestamp'].dt.date == coldest_day]
        
        # Calculate the max energy consumption in any 3-hour period on that day
        day_data = day_data.set_index('Timestamp')
        day_data['Rolling_Energy_Consumed_3h'] = day_data['Whole_System_Energy_Consumed'].rolling('3h').apply(lambda x: x[-1] - x[0], raw=True)
        max_consumption_3h = day_data['Rolling_Energy_Consumed_3h'].max()

        # Find the timestamp where this max consumption occurs
        max_3h_timestamp = day_data['Rolling_Energy_Consumed_3h'].idxmax()

        # Get the minimum internal temperature during this 3-hour period
        min_temp_coldest_3h = day_data.loc[max_3h_timestamp:max_3h_timestamp + pd.Timedelta(hours=3), 'Internal_Air_Temperature'].min()

        # Get the external temperature corresponding to the minimum internal temperature
        temp_ext_coldest_3h = day_data.loc[day_data['Internal_Air_Temperature'] == min_temp_coldest_3h, 'External_Air_Temperature'].iloc[0]

        # Extract the property ID from the filename (assuming it's in the format "Property_ID=XXXX.csv")
        property_id = filename.split('=')[1].replace('.csv', '')
        
        # Append the results to the list with additional information for the coldest moment
        results.append({
            'Property_ID_9': property_id, 
            'ColdestR_winter_Time_9': coldest_time, 
            'Max_Cons_3h_coldest_9': max_consumption_3h,
            'min_temp_coldest_3h_9': min_temp_coldest_3h,
            'temp_ext_coldest_3h_9': temp_ext_coldest_3h
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to an Excel file
output_file = "coldest_recorded_consumption_results_9.xlsx"
results_df.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}")



Skipping file Property_ID=EOH0303.csv as no autumn/winter data with non-zero consumption found.
Skipping file Property_ID=EOH0345.csv due to missing required columns.
Skipping file Property_ID=EOH0584.csv as no autumn/winter data with non-zero consumption found.
Skipping file Property_ID=EOH0955.csv as no autumn/winter data with non-zero consumption found.
Skipping file Property_ID=EOH1481.csv due to missing required columns.
Skipping file Property_ID=EOH1658.csv as no autumn/winter data with non-zero consumption found.
Skipping file Property_ID=EOH1880.csv due to missing required columns.
Skipping file Property_ID=EOH2196.csv as no autumn/winter data with non-zero consumption found.
Skipping file Property_ID=EOH2228.csv as no autumn/winter data with non-zero consumption found.
Skipping file Property_ID=EOH2334.csv as no autumn/winter data with non-zero consumption found.
Skipping file Property_ID=EOH2705.csv as no autumn/winter data with non-zero consumption found.
Results have been s